In [74]:
import numpy as np
import pandas as pd
import optuna

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score,f1_score


In [69]:
data = pd.read_csv("aqi.csv")

In [70]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87025 entries, 0 to 87024
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   87025 non-null  int64  
 1   StationId    87025 non-null  object 
 2   StationName  87025 non-null  object 
 3   City         87025 non-null  object 
 4   State        87025 non-null  object 
 5   Date         87025 non-null  object 
 6   PM2.5        83537 non-null  float64
 7   PM10         63064 non-null  float64
 8   NO           84796 non-null  float64
 9   NO2          85459 non-null  float64
 10  NOx          82470 non-null  float64
 11  NH3          57193 non-null  float64
 12  CO           84129 non-null  float64
 13  SO2          77492 non-null  float64
 14  O3           77427 non-null  float64
 15  Benzene      67238 non-null  float64
 16  Toluene      60701 non-null  float64
 17  Xylene       19441 non-null  float64
 18  AQI          87025 non-null  float64
 19  AQI_

In [71]:
(data.isnull().mean()*100).sort_values(ascending=False)

Xylene         77.660442
NH3            34.279805
Toluene        30.248779
PM10           27.533467
Benzene        22.737144
O3             11.029015
SO2            10.954323
NOx             5.234128
PM2.5           4.008044
CO              3.327779
NO              2.561333
NO2             1.799483
City            0.000000
StationName     0.000000
StationId       0.000000
Unnamed: 0      0.000000
Date            0.000000
State           0.000000
AQI             0.000000
AQI_Bucket      0.000000
dtype: float64

In [73]:
data["AQI_Bucket"].value_counts(normalize=True)*100

AQI_Bucket
Moderate        33.802930
Satisfactory    27.160011
Very Poor       13.515656
Poor            13.206550
Good             6.331514
Severe           5.983338
Name: proportion, dtype: float64

In [ ]:


# Drop useless / high-missing columns ONCE
data = data.drop(columns=["Unnamed: 0", "Xylene"])

# Target
TARGET = "AQI_Bucket"

X = data.drop(columns=[TARGET])
y = data[TARGET]


In [35]:
x_train, x_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    stratify=y,
    random_state=42
)


In [36]:
num_low = ['PM2.5', 'NO', 'NO2', 'CO']
num_med = ['PM10', 'NOx', 'SO2', 'O3']
cat_cols = ['StationName', 'City', 'State']

num_low_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='mean')),
    ('log', FunctionTransformer(np.log1p, validate=False))
])

num_med_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('log', FunctionTransformer(np.log1p, validate=False))
])

cat_pipe = Pipeline([
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(
        handle_unknown='ignore',
        sparse_output=False
    ))
])

preprocessor = ColumnTransformer([
    ('low', num_low_pipe, num_low),
    ('med', num_med_pipe, num_med),
    ('cat', cat_pipe, cat_cols)
])


In [39]:
def objective(trial):
    rf_params = {
        "n_estimators": trial.suggest_int("n_estimators", 300, 1000),
        "max_depth": trial.suggest_int("max_depth", 10, 40),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 10),
        "max_features": trial.suggest_categorical("max_features", ["sqrt", "log2"]),
        "class_weight": "balanced",
        "random_state": 42,
        "n_jobs": -1
    }

    model = Pipeline([
        ('preprocess', preprocessor),
        ('rf', RandomForestClassifier(**rf_params))
    ])

    cv = StratifiedKFold(
        n_splits=3,
        shuffle=True,
        random_state=42
    )

    score = cross_val_score(
        model,
        x_train,
        y_train,
        cv=cv,
        scoring='f1_weighted',
        n_jobs=-1
    ).mean()

    return score


In [40]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

print("Best CV F1:", study.best_value)
print("Best Parameters:")
for k, v in study.best_params.items():
    print(f"{k}: {v}")


[I 2026-01-30 21:29:00,585] A new study created in memory with name: no-name-1dec7b74-834f-45a1-8df9-fc01cfc58e20
[I 2026-01-30 21:30:17,994] Trial 0 finished with value: 0.7718520775489957 and parameters: {'n_estimators': 870, 'max_depth': 38, 'min_samples_split': 15, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 0 with value: 0.7718520775489957.
[I 2026-01-30 21:30:48,998] Trial 1 finished with value: 0.7735430241845056 and parameters: {'n_estimators': 372, 'max_depth': 30, 'min_samples_split': 9, 'min_samples_leaf': 2, 'max_features': 'log2'}. Best is trial 1 with value: 0.7735430241845056.
[I 2026-01-30 21:31:17,673] Trial 2 finished with value: 0.6651180300126166 and parameters: {'n_estimators': 472, 'max_depth': 10, 'min_samples_split': 6, 'min_samples_leaf': 8, 'max_features': 'log2'}. Best is trial 1 with value: 0.7735430241845056.
[I 2026-01-30 21:32:15,239] Trial 3 finished with value: 0.7247877670021948 and parameters: {'n_estimators': 684, 'max_depth': 12, '

Best CV F1: 0.7764183480603575
Best Parameters:
n_estimators: 311
max_depth: 22
min_samples_split: 17
min_samples_leaf: 1
max_features: sqrt


In [43]:
best_rf = RandomForestClassifier(
    **study.best_params,
    class_weight='balanced',
    random_state=42,
    n_jobs=-1
)

final_model = Pipeline([
    ('preprocess', preprocessor),
    ('rf', best_rf)
])

final_model.fit(x_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('rf', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('low', ...), ('med', ...), ...]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

In [44]:
y_pred = final_model.predict(x_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


Accuracy: 0.7729962654409652

Classification Report:

              precision    recall  f1-score   support

        Good       0.69      0.84      0.76      1102
    Moderate       0.83      0.75      0.79      5883
        Poor       0.61      0.72      0.66      2299
Satisfactory       0.81      0.80      0.80      4727
      Severe       0.81      0.84      0.82      1042
   Very Poor       0.80      0.76      0.78      2352

    accuracy                           0.77     17405
   macro avg       0.76      0.79      0.77     17405
weighted avg       0.78      0.77      0.77     17405

